In [ ]:
import itertools
import numpy as np
import networkx as nx

def find_compatible_dfa_single_button(seq_0,seq_1,num_states):
    """ Finds all DFAs with a single button (input symbol) of given state complexity (num_states) 
        that separate the sequences in seq_0 and seq_1.
    Args:
        seq_0: List of sequences that should lead to accepting states.
        seq_1: List of sequences that should lead to rejecting states.
        num_states: Number of states in the DFA.
    Returns:
        working_dfa: List of DFAs (as dictionaries) that separate seq_0 and seq_1.
    """
    working_dfa = []
    # generate all possible DFAs with num_states states and a single input symbol
    # for each DFA, check if it separates seq_0 and seq_1
    for a in itertools.product(range(num_states), repeat=num_states):
        dfa = dict(zip(range(num_states),a))
        init = 0
        out_0 = []
        for seq in seq_0:
            state = init
            for s in seq:
                state = dfa[state]
            out_0.append(state)
        out_1 = []
        
        for seq in seq_1:
            state = init
            for s in seq:
                state = dfa[state]
            out_1.append(state)

        intersection_fs = set(out_0).intersection(set(out_1))
        if len(intersection_fs) == 0:
            working_dfa.append(dfa)
    return working_dfa


def find_compatible_dfa_two_buttons(seq_0,seq_1,known_button,known_button_automata,num_states):
    """ Finds all compatible DFAs with two buttons (input symbols) of given state complexity (num_states) 
        that separate the sequences in seq_0 and seq_1, given the automata for one known button.
    Args:
        seq_0: List of sequences that should lead to accepting states.
        seq_1: List of sequences that should lead to rejecting states.
        known_button: The input symbol for which the automata transitions are known. E.g., 's'
        known_button_automata: A dictionary representing the automata transitions for the known button.
        num_states: Number of states in the DFA.
    Returns:
        working_dfa: List of DFAs (as dictionaries) that separate seq_0 and seq_1.
    """
    working_dfa = []
    for a in itertools.product(range(num_states), repeat=num_states):
        dfa = dict(zip(range(num_states),a))
        init = 0
        out_0 = []
        for seq in seq_0:
            state = init
            for s in seq:
                if s == known_button:
                    state = known_button_automata[state]
                else:
                    state = dfa[state]
            out_0.append(state)
        out_1 = []
        for seq in seq_1:
            state = init
            for s in seq:
                if s == known_button:
                    state = known_button_automata[state]
                else:
                    state = dfa[state]
            out_1.append(state)

        intersection_fs = set(out_0).intersection(set(out_1))
        if len(intersection_fs) == 0:
            working_dfa.append(dfa)
            
    return working_dfa


def get_equivalent_classes_isomorphic(list_of_automata):
    """ Given a list of DFAs (as dictionaries), finds the equivalence classes of isomorphic DFAs.
        We say two DFAs are isomorphic if they are isomorphic as directed graphs (which they are).
    Args:
        list_of_automata: List of DFAs (as dictionaries).
    Returns:
        equivalent_classes: List of indices representing the equivalence class of each DFA.
        unique_aut: List of unique DFAs (as dictionaries) representing each equivalence class.
    """
    def get_adjecency_matrix(automaton):
        adj = np.zeros((len(automaton),len(automaton)))
        for i in automaton.keys():
            adj[i,automaton[i]] = 1
            adj[automaton[i],i] = 1
        return adj

    def is_isomorphic(automaton1,automaton2):
        g1 = nx.from_numpy_array(get_adjecency_matrix(automaton1))
        g2 = nx.from_numpy_array(get_adjecency_matrix(automaton2))
        return nx.is_isomorphic(g1,g2)
    
    equivalent_classes = [0]*len(list_of_automata)
    unique_aut = []
    unique_aut.append(list_of_automata[0])
    for i in range(1,len(list_of_automata)):
        found = False
        for j in range(i):
            if is_isomorphic(list_of_automata[i],list_of_automata[j]):
                equivalent_classes[i] = equivalent_classes[j]
                found = True
        if not found:
            equivalent_classes[i] = i
            unique_aut.append(list_of_automata[i])
    return equivalent_classes, unique_aut

In the following example, we want to test the hypothesis that there are no DFAs of dimension smaller than 6 that can separate the given sequences. These sequences are precisely those that produce a deterministic output in quantum experiments involving `s`-gate, which is $\sqrt{Z}$, and the `h`-gate, the Hadamard gate, applied to a $|+\rangle$-state and then measured in the $X$ basis.

We denote by `seq_0` the set of sequences that ma p our quantum state back to $|+\rangle$, and by `seq_1` the set of sequences that map it to $|-\rangle$. 

We confirm that there are no DFAs of dimension smaller than 6 that can separate these sequences by iterating over all possible DFAs of dimension 6, and trying one by one wether they can separate the sequences. It turns that exactly one of these DFAs can separate the sequences! Note that the set of all DFAs of dimension 6 includes all DFAs of smaller dimension as well, so if there were a DFA of smaller dimension that could separate the sequences, we would have found it in this search.

In [ ]:
# Find all s-gate DFAs of dimension 6 that separate the s-gate sequences
# running this cell may take some time (about 20 minutes on my machine)
dfa_dim = 6

seq_0 = ['','ssss', 'ssssssss']
seq_1 = ['ss','ssssss']

s_automata = find_compatible_dfa_single_button(seq_0,seq_1, dfa_dim)
eqs,unqs = get_equivalent_classes_isomorphic(s_automata)

In [ ]:
# Next, include also sequence involving h-gates.
# Following are subsets of sequences defined for the s and h case in https://arxiv.org/pdf/2510.09575 Eq. (9).
# These are enough to confirm our hypothesis and are found via considering all pairs of group elements in the single-qubit
# Clifford group that would map |+> to |+> and |->. 
# Note however that these sequences are not minimal, i.e., some of them could be removed while still keeping the same set of valid automata!

seq_0_all = ['', 'hh', 'hsh', 'shs', 'hssh', 'hhhh', 'ssss', 'hhhsh', 'hsssh', 'hshhh', 'hhshs', 'shshh', 'shshsh', 'hshshs', 'hhhssh', 'hsshhh', 'sshhss', 'hssssh', 'ssshhs', 'shsshs', 'hshhsh', 'shsssss', 'hsssssh', 'hsshshs', 'sssshsh', 'hshhssh', 'ssshsss', 'ssssshs', 'shshssh', 'sshshss', 'hsshhsh', 'hsshhssh', 'sssshssh', 'sshsshss', 'shshsssh', 'hsshssss', 'shsshshsh', 'ssshhsshs', 'sshssshss', 'shsshsshs', 'ssshshhss', 'shssssshsh', 'ssshssshsh', 'shsshshssh', 'sshshssshs', 'ssshsssshs', 'ssshshshss', 'sshsssshss', 'sshshsshsh', 'hsshshsshs', 'sshssssshss', 'sshsshssshs', 'hsshsssshsh', 'ssshshsshss', 'sshsshsshsh', 'sshshsshssh', 'ssshssshssh', 'ssshshssshss', 'sshsshsshssh']
seq_1_all = ['ss', 'sshh', 'hhss', 'shhs', 'sshsh', 'ssshs', 'hshss', 'shsss', 'ssssss', 'hsshss', 'sshssh', 'hhssshs', 'shshhss', 'sshsssh', 'shsshhs', 'ssshshh', 'sshshhh', 'hhsshsh', 'hssshss', 'shhsshs', 'hhsshssh', 'shsssshs', 'hshsshsh', 'sshssssh', 'shshshss', 'sshshshs', 'hsssshss', 'shssshsh', 'sshsshhh', 'hshssshs', 'sshshhsh', 'shsshsss', 'ssshshsh', 'ssshsshs', 'shshsshss', 'sshsshhsh', 'hssssshss', 'sshsssssh', 'hsshssshs', 'sshsshshs', 'ssshsssss', 'hsshsshsh', 'sssssshsh', 'ssshshssh', 'sshshhssh', 'shssshssh', 'hshsshssh', 'sshsshhssh', 'ssshshsssh', 'shshssshss', 'shsshhsshs', 'hsshsshssh', 'ssshsshsshs', 'shsshssshsh', 'ssshssssshsh']

unique_h_automata = []
for sauto in unqs:
    out = find_compatible_dfa_two_buttons(seq_0_all,seq_1_all,'s',sauto, dfa_dim)
    if len(out)>0:
        eqh, unqh = get_equivalent_classes_isomorphic(out)
        unique_h_automata.append(unqh)
    else:
        unique_h_automata.append(out)

print("possible s-automaton up to isomorphism:", len(unqs))
print("possible h-automaton per s-automaton:", [len(ua) for ua in unique_h_automata])
print("There seems to be only one valid automaton pair (s,h)! Which is exactly the one isomorphic to the quantum example:")
for i in range(len(unqs)):
    if len(unique_h_automata[i])>0:
        print("s-automaton:", unqs[i])
        print("h-automaton(s):", unique_h_automata[i])


possible s-automaton up to isomorphism: 11
possible h-automaton per s-automaton: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
There seems to be only one valid automaton pair (s,h)! Which is exactly the one isomorphic to the quantum example:
s-automaton: {0: 1, 1: 2, 2: 3, 3: 0, 4: 4, 5: 5}
h-automaton(s): [{0: 4, 1: 3, 2: 5, 3: 1, 4: 0, 5: 2}]
